# 🐼 Pandas Integration - Data Analysis with Bright Data SDK

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/vzucher/brightdata-sdk-python/blob/master/notebooks/02_pandas_integration.ipynb)

Learn how to integrate Bright Data SDK with pandas for powerful data analysis.

## What You'll Learn
1. Converting results to DataFrames
2. Batch scraping to DataFrame
3. Data cleaning and analysis
4. Exporting to CSV/Excel
5. Visualization with matplotlib

---


## 📦 Setup


In [1]:
# Install required packages
%pip install brightdata-sdk pandas matplotlib seaborn python-dotenv -q

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from brightdata import BrightDataClient

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ All packages loaded")


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All packages loaded


In [2]:
# Authentication - Load from .env file (recommended)
from dotenv import load_dotenv
load_dotenv()

API_TOKEN = os.getenv('BRIGHTDATA_API_TOKEN')
# Or set directly: API_TOKEN = "your_api_token_here"

if not API_TOKEN:
    raise ValueError("Set BRIGHTDATA_API_TOKEN in .env file or directly above")

client = BrightDataClient(token=API_TOKEN)
print(f"✅ Client initialized (token: {API_TOKEN[:10]}...)")

✅ Client initialized (token: 7011787d-2...)


## 📊 Method 1: Single Result to DataFrame

Convert a single scrape result to a DataFrame:


In [10]:
# Scrape one product (async required in Jupyter)
async with client.scrape.amazon.engine:
    result = await client.scrape.amazon.products(
        url="https://www.amazon.com/dp/B0CRMZHDG8",
        timeout=660
    )

# Convert to DataFrame
if result.success and result.data:
    df = pd.DataFrame([result.data])
    
    # Add metadata
    df['url'] = result.url
    df['cost'] = result.cost
    df['scraped_at'] = pd.Timestamp.now()
    
    # Show all available fields
    print(f"✅ DataFrame created: {len(df.columns)} fields available\n")
    print("📋 All available fields:")
    print("-" * 50)
    for key, value in result.data.items():
        val_str = str(value)[:50] + "..." if len(str(value)) > 50 else str(value)
        print(f"  {key}: {val_str}")
    print("-" * 50)
    print(f"\n💡 Note: {len(result.data)} fields available. Examples below use 5 key fields:")
    print("   title, rating, reviews_count, price, cost\n")
    
    display(df[['title', 'rating', 'reviews_count', 'url', 'cost']].head())
else:
    print(f"❌ Failed: {result.error}")

✅ DataFrame created: 76 fields available

📋 All available fields:
--------------------------------------------------
  title: STANLEY Quencher H2.0 Tumbler with Handle and Stra...
  seller_name: None
  brand: STANLEY
  description: Constructed of recycled stainless steel for sustai...
  currency: USD
  reviews_count: 2227
  categories: ['Home & Kitchen', 'Kitchen & Dining', 'Storage & ...
  parent_asin: B0CRMZHDG8
  asin: B0CRMZHDG8
  number_of_sellers: 1
  root_bs_rank: 16399
  answered_questions: 0
  domain: https://www.amazon.com/
  images_count: 9
  url: https://www.amazon.com/STANLEY-Flowstate-3-Positio...
  video_count: 6
  image_url: https://m.media-amazon.com/images/I/61Q4eGZWFSL._A...
  item_weight: 1.43 Pounds
  rating: 4.7
  product_dimensions: 10"W x 13.25"H
  seller_id: ATVPDKIKX0DER
  image: https://m.media-amazon.com/images/I/61Q4eGZWFSL._A...
  date_first_available: March 11, 2024
  model_number: Stanley Quencher H2.O FlowState™ Tumbler 40 oz Fuc...
  manufacturer: Stan

,title,rating,reviews_count,url,cost
0,STANLEY Quencher H2.0 Tumbler with Handle and ...,4.7,2227,https://www.amazon.com/dp/B0CRMZHDG8,0.001


## 🔄 Method 2: Batch Scraping to DataFrame

Scrape multiple URLs and create a comprehensive DataFrame:


---
### 💡 Pro Tips for Large-Scale Scraping

#### 1. Trigger-Then-Poll Pattern
```python
# Trigger all jobs first (fast)
jobs = {}
for url in urls:
    job = await client.scrape.amazon.products_trigger(url=url)
    jobs[url] = job

# Then poll in parallel (efficient)
results = await asyncio.gather(*[poll_job(url, job) for url, job in jobs.items()])
```

#### 2. Rate Limiting (Built-in!)
The SDK automatically handles rate limiting - no need to add delays!

#### 3. Memory Management
```python
# For very large batches, write incrementally
for url in urls:
    result = await scrape(url)
    pd.DataFrame([result]).to_csv('results.csv', mode='a', header=False)
```

#### 4. Async Context Manager
```python
# Always use async with in Jupyter
async with client.scrape.amazon.engine:
    result = await client.scrape.amazon.products(url=url)
```

---


In [7]:
# List of Amazon product URLs
urls = [
    "https://www.amazon.com/dp/B0CRMZHDG8",
    "https://www.amazon.com/dp/B0D4C69XG2",
    "https://www.amazon.com/dp/B0CX23V2ZK",
]

import asyncio

async def poll_job(url, job):
    """Poll single job, return (url, result) or (url, None) if pending."""
    status = await job.status()
    if status == 'ready':
        data = await job.fetch()
        item = data[0] if isinstance(data, list) and data else {}
        return url, {
            'url': url,
            'title': item.get('title', 'N/A'),
            'price': item.get('final_price', item.get('final_price_high', 'N/A')),
            'rating': item.get('rating', 'N/A'),
            'reviews_count': item.get('reviews_count', 0),
            'cost': job.cost_per_record,
            'status': 'success'
        }
    elif status in ('error', 'failed'):
        return url, {'url': url, 'error': f'Job failed: {status}', 'status': 'failed'}
    return url, None

print(f"🚀 Triggering {len(urls)} scrapes...")
triggered_jobs = {}

# Step 1: Trigger all scrapes (fast)
async with client.scrape.amazon.engine:
    for i, url in enumerate(urls, 1):
        print(f"  [{i}/{len(urls)}] Triggering: {url[-15:]}...")
        try:
            job = await client.scrape.amazon.products_trigger(url=url)
            triggered_jobs[url] = job
            print(f"       ✅ Triggered: {job.snapshot_id}")
        except Exception as e:
            print(f"       ❌ Failed to trigger: {e}")

print(f"\n⏳ Polling {len(triggered_jobs)} jobs in parallel...")

# Step 2: Poll all jobs in parallel until complete
results = []
pending_jobs = dict(triggered_jobs)
max_wait = 660
poll_interval = 10
elapsed = 0

async with client.scrape.amazon.engine:
    while elapsed < max_wait and pending_jobs:
        print(f"  Checking {len(pending_jobs)} jobs... ({elapsed}s elapsed)")
        
        poll_results = await asyncio.gather(*[
            poll_job(url, job) for url, job in pending_jobs.items()
        ])
        
        for url, result in poll_results:
            if result:
                results.append(result)
                del pending_jobs[url]
                icon = "✅" if result['status'] == 'success' else "❌"
                print(f"       {icon} {result.get('title', url)[:35]}...")
        
        if pending_jobs:
            await asyncio.sleep(poll_interval)
            elapsed += poll_interval

# Handle timeouts
for url in pending_jobs:
    results.append({'url': url, 'error': 'Timeout', 'status': 'failed'})

# Create DataFrame
df_batch = pd.DataFrame(results)
print(f"\n✅ Scraped {len(df_batch)} products")
print(f"   Success: {(df_batch['status'] == 'success').sum()}")
print(f"   Failed: {(df_batch['status'] != 'success').sum()}")

# Show failures if any
failed = df_batch[df_batch['status'] == 'failed']
if len(failed) > 0:
    print("\n⚠️  Failed URLs:")
    for _, row in failed.iterrows():
        print(f"   - {row['url']}: {row.get('error', 'Unknown')}")

🚀 Triggering 3 scrapes...
  [1/3] Triggering: m/dp/B0CRMZHDG8...
       ✅ Triggered: sd_mkzex21m14kkunc5hn
  [2/3] Triggering: m/dp/B0D4C69XG2...
       ✅ Triggered: sd_mkzex2hvf2baotbxu
  [3/3] Triggering: m/dp/B0CX23V2ZK...
       ✅ Triggered: sd_mkzex2zlbchdsc7f1

⏳ Polling 3 jobs in parallel...
  Checking 3 jobs... (0s elapsed)
  Checking 3 jobs... (10s elapsed)
       ✅ STANLEY Quencher H2.0 Tumbler with ...
       ✅ Jellycat Amuseables Table Tennis Pa...
       ✅ Apple 2024 MacBook Air 13-inch Lapt...

✅ Scraped 3 products
   Success: 3
   Failed: 0


In [8]:
display(df_batch.head())

# Summary statistics
if 'cost' in df_batch.columns:
    total_cost = df_batch['cost'].dropna().sum()
    print("\n📊 Summary:")
    print(f"Total cost: ${total_cost:.4f}")

,url,title,price,rating,reviews_count,cost,status
0,https://www.amazon.com/dp/B0CRMZHDG8,STANLEY Quencher H2.0 Tumbler with Handle and ...,NaN,4.7,2227,0.001,success
1,https://www.amazon.com/dp/B0D4C69XG2,"Jellycat Amuseables Table Tennis Paddle Toy, 1...",50.0,4.7,664,0.001,success
2,https://www.amazon.com/dp/B0CX23V2ZK,Apple 2024 MacBook Air 13-inch Laptop with M3 ...,NaN,4.8,1047,0.001,success



📊 Summary:
Total cost: $0.0030


## 💾 Export Data


In [9]:
# Export to CSV
df_batch.to_csv('amazon_products.csv', index=False)
print("✅ Exported to amazon_products.csv")

# Export to Excel (requires openpyxl)
try:
    df_batch.to_excel('amazon_products.xlsx', index=False, sheet_name='Products')
    print("✅ Exported to amazon_products.xlsx")
except ImportError:
    print("⚠️  Install openpyxl for Excel export: pip install openpyxl")

✅ Exported to amazon_products.csv
⚠️  Install openpyxl for Excel export: pip install openpyxl


## ✅ Summary

You learned:
- ✅ Converting SDK results to DataFrames
- ✅ Batch scraping workflows (async)
- ✅ Exporting to CSV/Excel

